In [7]:
from datasets import load_dataset
import nltk
nltk.download('punkt')


model_name = "google/t5-v1_1-small"
dataset_name = "kmfoda/booksum"

# Load the dataset
dataset = load_dataset(dataset_name)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emmanuelmoran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-small")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [9]:
def preprocess_data(examples):
    inputs = [ f'summarize the following chapter: {example}\nsummary:' for example in examples['chapter']]
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/1484 [00:00<?, ? examples/s]

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-booksum-pgc",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)


d:\OTROS\LLMs\llm-venv\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [12]:
trainer.train()

  0%|          | 0/24000 [00:00<?, ?it/s]

In [ ]:
# from huggingface_hub import login

# login()

In [ ]:
# model.push_to_hub("t5-small-ft-pgc")
# tokenizer.push_to_hub("t5-small-ft-pgc")